<a href="https://colab.research.google.com/github/RahulParajuli/OpenMap_NETWORKx_OSMnx/blob/main/extractingMAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# print(column_list)
!pip install osmnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 95 kB 3.0 MB/s 
     |████████████████████████████████| 11.2 MB 23.4 MB/s 
     |████████████████████████████████| 1.0 MB 58.8 MB/s 
     |████████████████████████████████| 62 kB 1.7 MB/s 
     |████████████████████████████████| 1.0 MB 56.2 MB/s 
     |████████████████████████████████| 6.3 MB 43.4 MB/s 
     |████████████████████████████████| 16.7 MB 39.9 MB/s 
     |████████████████████████████████| 944 kB 55.2 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. T

In [2]:
import IPython
import warnings
warnings.filterwarnings('ignore')

In [3]:
import folium
import osmnx as ox
import networkx as nx

ox.config(use_cache=True, log_console=True)

place = ['Kathmandu, Nepal', 'Bhaktapur, Nepal', 'Lalitpur, Nepal']
G = ox.graph_from_place((place), network_type='drive')

G = ox.speed.add_edge_speeds(G)
G = ox.speed.add_edge_travel_times(G)

agents = [{
    "lat": 85000.56,
    "lng": 45000.6776,
    "agent_id": 1,
    "agent_name" : "",
    "agent_ph" : "",
},
    {
    "lat": 45.56,
    "lng": 45.6776,
    "agent_id": 2,
    "agent_name" : "",
    "agent_ph" : "",
},
    {
    "lat": 50.56,
    "lng": 50.6776,
    "agent_id": 3,
    "agent_name" : "",
    "agent_ph" : "",
},
]

for agent in agents:
  a = (agent['lat'])
  b = (agent['lng'])
  rider_location = ox.get_nearest_node(G, (a , b) , method='balltree')

orig = ox.get_nearest_node(G, (27.7189016, 85.3096064), method='balltree')
dest = ox.get_nearest_node(G, (27.7132141, 85.3245858))

"""#imputing travel speed and times"""
G = ox.add_edge_speeds(G)
  # calculate travel time (seconds) for all edges
G = ox.add_edge_travel_times(G)
  # see mean speed/time values by road type
edges = ox.graph_to_gdfs(G, nodes=False)
edges['highway'] = edges['highway'].astype(str)
edges.groupby('highway')[['length', 'speed_kph', 'travel_time']].mean().round(1)

hwy_speeds = {'residential': 35,
                'secondary': 50,
                'tertiary': 60}
G = ox.add_edge_speeds(G, hwy_speeds)
G = ox.add_edge_travel_times(G)
route1 = nx.shortest_path(G, orig, dest, weight='length')
route2 = nx.shortest_path(G, orig, dest, weight='travel_time')

route1_length = int(sum(ox.utils_graph.get_route_edge_attributes(G, route1, 'length')))
route2_length = int(sum(ox.utils_graph.get_route_edge_attributes(G, route2, 'length')))
route1_time = int(sum(ox.utils_graph.get_route_edge_attributes(G, route1, 'travel_time')))
route2_time = int(sum(ox.utils_graph.get_route_edge_attributes(G, route2, 'travel_time')))
route1_time_minute = float(route1_time/60)
route2_time_minute = float(route2_time/60)
print('Route 1 is', route1_length, 'meters and takes', route1_time, 'seconds.'," which is :", route1_time_minute, "minutes")
print('Route 2 is', route2_length, 'meters and takes', route2_time, 'seconds.',' which is :', route2_time_minute, "minutes")

route = nx.shortest_path(G, orig, dest, 'travel_time')
route_map = ox.plot_route_folium(G, route)
route_map.save('test2.html')

Route 1 is 1988 meters and takes 205 seconds.  which is : 3.4166666666666665 minutes
Route 2 is 2208 meters and takes 181 seconds.  which is : 3.0166666666666666 minutes
